# Conteúdo final sobre Pandas.

## Aprendendo a usar `melt`, `pivot` e `pivot_table`.

![Exemplo](https://pandas.pydata.org/docs/_images/reshaping_pivot.png)

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/BindiChen/machine-learning/main/data-analysis/003-pandas-pivot-table/data/coffee_sales.csv')

In [3]:
df.head()

,order_date,market,region,product_category,product,cost,inventory,net_profit,sales
0,2010-01-01,Wholesale,Central,Coffee,Amaretto,89,777,94,219
1,2010-01-01,Wholesale,Central,Coffee,Columbian,83,623,68,190
2,2010-01-01,Wholesale,Central,Coffee,Decaf Irish Cream,95,821,101,234
3,2010-01-01,Wholesale,Central,Tea,Green Tea,44,623,30,100
4,2010-01-01,Wholesale,Central,Espresso,Caffe Mocha,54,456,54,134


Perceba o problema quando temos valores duplicados de linhas e colunas quando fazemos pivot...

De fato, o pivot está quase sempre associado com o groupby.

Para a nossa felicidade, existe uma função que faz o pivot e a agregação ao mesmo tempo, chamado `pd.pivot_table`.

In [4]:
# Método mais simples 

df.pivot_table(index='region')

,cost,inventory,net_profit,sales
region,,,,
Central,84.157738,742.648810,69.830357,197.206101
East,85.421171,778.191441,66.685811,201.099099
South,65.327381,478.273810,48.330357,154.651786
West,93.608631,872.632440,55.056548,202.577381


O que aconteceu nesse exemplo?

In [5]:
# Usando múltiplos índices 

df.pivot_table(index=['region','product_category'])

cost   inventory  net_profit       sales
region  product_category                                                
Central Coffee             74.390625  680.781250   60.583333  179.895833
        Espresso           89.666667  882.090278   81.600694  207.295139
        Herbal tea         87.065476  763.952381   73.681548  202.047619
        Tea                87.690476  672.529762   66.458333  203.500000
East    Coffee            110.976190  580.785714  184.476190  337.148810
        Espresso           95.600000  767.133333   26.016667  201.654167
        Herbal tea        101.240741  963.212963   29.736111  191.486111
        Tea                46.962121  762.484848   58.931818  121.882576
South   Coffee             78.916667  803.687500   60.947917  173.223958
        Espresso           62.305556  184.645833   52.100694  156.215278
        Herbal tea         56.270833  593.302083   30.057292  133.734375
West    Coffee             90.814103  931.583333   27.964744  185.410256
        Espresso           84.133333  828.933333   66.305556  194.188889
        Herbal tea         98.987179  802.282051   84.304487  231.692308
        Tea               100.844444  926.211111   41.938889  200.611111

In [7]:
# E se quisermos apenas a coluna net_profit? 

df.pivot_table(index=['region','product_category'], values='net_profit')

# Por que não usamos o valor de columns?

net_profit
region  product_category            
Central Coffee             60.583333
        Espresso           81.600694
        Herbal tea         73.681548
        Tea                66.458333
East    Coffee            184.476190
        Espresso           26.016667
        Herbal tea         29.736111
        Tea                58.931818
South   Coffee             60.947917
        Espresso           52.100694
        Herbal tea         30.057292
West    Coffee             27.964744
        Espresso           66.305556
        Herbal tea         84.304487
        Tea                41.938889

In [8]:
# Outra forma para mostrar o mesmo resultado.

df.pivot_table(index='region', columns='product_category', values='net_profit')

product_category,Coffee,Espresso,Herbal tea,Tea
region,,,,
Central,60.583333,81.600694,73.681548,66.458333
East,184.476190,26.016667,29.736111,58.931818
South,60.947917,52.100694,30.057292,NaN
West,27.964744,66.305556,84.304487,41.938889


In [9]:
# Perceba que a agregação utilizada aqui é automática. E se quisermos a soma de lucro, ao invés da média? Neste caso, passamos a função de agregação (neste caso, apenas np.sum)

df.pivot_table(index='region', columns='product_category', values='net_profit', aggfunc=np.sum)

product_category,Coffee,Espresso,Herbal tea,Tea
region,,,,
Central,23264.0,23501.0,24757.0,22330.0
East,30992.0,6244.0,6423.0,15558.0
South,11702.0,15005.0,5771.0,NaN
West,8725.0,23870.0,26303.0,15098.0


In [10]:
df.pivot_table(index=['region'], values=['sales', 'cost'], aggfunc='sum', columns=['product_category'], fill_value=0)

cost                             sales                      \
product_category Coffee Espresso Herbal tea    Tea Coffee Espresso Herbal tea   
region                                                                          
Central           28566    25824      29254  29464  69080    59701      67888   
East              18644    22944      21868  12398  56641    48397      41361   
South             15152    17944      10804      0  33259    44990      25677   
West              28334    30288      30884  36304  57848    69908      72288   

                         
product_category    Tea  
region                   
Central           68376  
East              32177  
South                 0  
West              72220

In [12]:
# Forma um pouco melhor...
df.pivot_table(index=['region', 'product_category'], values=['sales', 'cost'], aggfunc='sum', margins=True)

cost   sales
region  product_category                
Central Coffee             28566   69080
        Espresso           25824   59701
        Herbal tea         29254   67888
        Tea                29464   68376
East    Coffee             18644   56641
        Espresso           22944   48397
        Herbal tea         21868   41361
        Tea                12398   32177
South   Coffee             15152   33259
        Espresso           17944   44990
        Herbal tea         10804   25677
West    Coffee             28334   57848
        Espresso           30288   69908
        Herbal tea         30884   72288
        Tea                36304   72220
All                       358672  819811

In [15]:
df.pivot_table(index=['region','product_category'],columns='order_date',values=['sales'], aggfunc='sum')

sales                                   \
order_date               2010-01-01 2010-02-01 2010-03-01 2010-04-01   
region  product_category                                               
Central Coffee                 2663       2708       2703       2815   
        Espresso               2370       2432       2467       2446   
        Herbal tea             2664       2704       2691       2735   
        Tea                    2649       2659       2702       2774   
East    Coffee                 2105       2061       2126       2258   
        Espresso               1853       1966       1907       1935   
        Herbal tea             1609       1621       1638       1753   
        Tea                    1213       1272       1250       1267   
South   Coffee                 1207       1260       1265       1280   
        Espresso               1770       1803       1781       1790   
        Herbal tea              999       1019       1009        995   
West    Coffee                 2339       2298       2313       2332   
        Espresso               2723       2759       2796       2798   
        Herbal tea             2602       2702       2739       2884   
        Tea                    2789       2828       2858       2881   

                                                                      \
order_date               2010-05-01 2010-06-01 2010-07-01 2010-08-01   
region  product_category                                               
Central Coffee                 2878       3008       3148       3012   
        Espresso               2497       2497       2556       2554   
        Herbal tea             2795       2908       2976       2961   
        Tea                    2840       2863       2912       2909   
East    Coffee                 2347       2625       2735       2673   
        Espresso               1896       2071       1992       1984   
        Herbal tea             1779       1795       1926       1677   
        Tea                    1319       1404       1395       1469   
South   Coffee                 1369       1429       1489       1579   
        Espresso               1849       1896       1923       1923   
        Herbal tea              978       1016       1036       1139   
West    Coffee                 2351       2495       2541       2523   
        Espresso               2829       2897       2992       3077   
        Herbal tea             2965       3133       3380       3386   
        Tea                    3000       3088       3160       3163   

                                                ...                        \
order_date               2010-09-01 2010-10-01  ... 2011-03-01 2011-04-01   
region  product_category                        ...                         
Central Coffee                 2734       2694  ...       2881       3000   
        Espresso               2394       2391  ...       2628       2607   
        Herbal tea             2747       2734  ...       2867       2915   
        Tea                    2851       2762  ...       2879       2956   
East    Coffee                 2362       2094  ...       2265       2407   
        Espresso               1887       2124  ...       2031       2062   
        Herbal tea             1539       1544  ...       1745       1868   
        Tea                    1337       1339  ...       1331       1351   
South   Coffee                 1389       1341  ...       1348       1365   
        Espresso               1844       1820  ...       1898       1908   
        Herbal tea             1033       1119  ...       1075       1060   
West    Coffee                 2359       2234  ...       2464       2485   
        Espresso               2816       2857  ...       2979       2982   
        Herbal tea             2850       2879  ...       2919       3075   
        Tea                    2950       2917  ...       3045       3071   

                                                 

Usando unstack:

![Exemplo](https://pandas.pydata.org/docs/_images/reshaping_unstack.png)

## Exercícios

Observe a base de dados sobre venda de [veículos](https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho/code?select=CAR+DETAILS+FROM+CAR+DEKHO.csv). Usando o conhecimento aprendido até agora, gere visualizações e análises que sejam pertinentes à base de dados. 

**Importante!** Crie pelo menos uma análise (obrigatória) em função das fabricantes de carro.